In [ ]:
import numpy as np
# Assuming timtiffread and smooth_image_and_subtract_background7 are implemented elsewhere

class PRISM_Data():
    def __init__(self, filedir, channels):
        self.filedir = filedir  # holds the file list
        self.curFile = ""  # holds the name of the current file
        self.fileIdx = 0  # holds the index of the current file in fList
        self.img = None  # holds the raw image/stack data of the current file
        self.smooth = None  # holds the smoothed image/stack data of the current file
        # self.isMovie = 0  # 1 if files in the list are movies, 0 if files are individual images
        # self.nFrames = 0  # number of frames in current file (if movie)
        # self.curFrame = 0  # current frame in current file (if movie)


    def setFListFromParams(self, params):
        self.reset()
        # This method needs to be adapted based on the data loading strategy in Python
        # The MATLAB version uses directory and file existence checks and file listing,
        # which can be achieved in Python using os.path and os.listdir or glob.glob
        # For simplicity, this is left as a placeholder

    def setFList(self, fList):
        self.reset()
        if isinstance(fList, list):
            self.fList = fList
        else:
            print("Input file list has wrong format; resetting airlocalize data object.")

    def getFList(self):
        return self.fList

    def setFileIdx(self, idx):
        if idx > len(self.fList) or idx <= 0:
            print(f"Cannot access file index {idx}; file list has {len(self.fList)} entries.")
            return
        if idx != self.fileIdx:
            self.resetCurrentFile()
            self.fileIdx = idx
            self.curFile = self.fList[idx - 1]  # -1 for zero-based indexing in Python
        else:
            if self.curFile != self.fList[idx - 1]:
                self.resetCurrentFile()
                self.fileIdx = idx
                self.curFile = self.fList[idx - 1]

    def getFileIdx(self):
        return self.fileIdx

    def setCurFile(self, fileName):
        if fileName not in self.fList:
            print(f"Desired file {fileName} is not part of existing file list; cannot set as current.")
        else:
            idx = self.fList.index(fileName) + 1  # +1 to match MATLAB's one-based indexing
            self.setFileIdx(idx)

    def getCurFile(self):
        return self.curFile

    def setNFrames(self):
        # This method depends on how the image data is loaded and handled in Python
        pass

    def setCurFrame(self, newFrame):
        # Similar to setNFrames, needs adaptation based on Python image handling
        pass

    def isFileIndexImgLoaded(self, idx):
        # Check if an image for a given index is loaded; adapt based on Python logic
        pass

    def retrieveImg(self, overwrite):
        # Adapt based on how images are loaded and managed in Python
        pass

    def retrieveSmoothedImg(self, params, overwrite):
        # Adapt based on image smoothing and background subtraction in Python
        pass

In [ ]:
import numpy as np
from scipy.ndimage import gaussian_filter, generate_binary_structure, binary_dilation
from scipy import ndimage

def find_isolated_maxima(alData, params, verbose=True):
    """
    Finds local maxima above the threshold in the smoothed image.

    Parameters:
    - alData: An object-like structure with the attributes 'smooth', 'img', 'isMovie', 'curFrame'.
    - params: An object-like structure with 'threshLevel', 'threshUnits', 'minDistBetweenSpots', 'numDim', 'maxSpots'.
    - verbose: If True, prints additional information.
    
    Returns:
    - maxima: A list of local maxima coordinates and their intensities.
    """
    # Initialize maxima
    maxima = []

    # Determine the threshold intensity based on the specified units
    if params.threshUnits == 'absolute':
        threshInt = params.threshLevel
    elif params.threshUnits in ['SD', 'legacySD']:
        if not alData.isMovie:
            img = alData.img if params.threshUnits == 'SD' else alData.smooth
            threshInt = params.threshLevel * np.std(img)
        else:
            img = alData.img[:,:,alData.curFrame] if params.threshUnits == 'SD' else alData.smooth[:,:,alData.curFrame]
            threshInt = params.threshLevel * np.std(img)
    else:
        raise ValueError("Unsupported threshold units.")

    if verbose:
        print(f"Threshold value is {threshInt} in {params.threshUnits} units.")

    # Find local maxima
    smooth_img = alData.smooth if not alData.isMovie else alData.smooth[:,:,alData.curFrame]
    footprint = generate_binary_structure(params.numDim, 1)
    footprint = ndimage.iterate_structure(footprint, params.minDistBetweenSpots)
    local_max = ((smooth_img == ndimage.maximum_filter(smooth_img, footprint=footprint)) & (smooth_img > threshInt))

    # Extract coordinates and intensities of the local maxima
    coordinates = np.argwhere(local_max)
    intensities = smooth_img[local_max]

    if params.numDim == 2: maxima = np.column_stack((coordinates, intensities))
    elif params.numDim == 3: maxima = np.column_stack((coordinates, intensities))
    
    # Sort maxima by intensity in descending order and truncate if necessary
    maxima = maxima[maxima[:, -1].argsort()[::-1]]
    if len(maxima) > params.maxSpots:
        maxima = maxima[:params.maxSpots]

    if verbose:
        print(f"Detected {len(maxima)} spots.")

    return maxima


## predetection of local maxima

In [ ]:
import numpy as np
from scipy import ndimage
from scipy.ndimage import gaussian_filter, generate_binary_structure, binary_dilation

def find_isolated_maxima(alData, params, verbose=False):
    """
    Finds local maxima above the threshold in the smoothed image.

    alData: dict containing 'smooth', 'img', 'isMovie', and 'curFrame' keys.
    params: dict containing 'threshLevel', 'threshUnits', 'minDistBetweenSpots', 'numDim', and 'maxSpots'.
    verbose: boolean, prints additional information if True.

    Example usage:
        alData = {
            'smooth': your_smoothed_image_here,
            'img': your_original_image_here,
            'isMovie': False,  # or True if it's a movie
            'curFrame': 0  # current frame to process if it's a movie
        }
        params = {
            'threshLevel': your_threshold_level,
            'threshUnits': 'absolute',  # or 'SD' or 'legacySD'
            'minDistBetweenSpots': your_minimum_distance_between_spots,
            'numDim': 2,  # or 3 for 3D images
            'maxSpots': your_maximum_number_of_spots_allowed
        }
        maxima = find_isolated_maxima_clean3(alData, params, verbose=True)
    """
    
    # Retrieve threshold level
    if params['threshUnits'] == 'absolute':
        threshInt = params['threshLevel']
    elif params['threshUnits'] in ['SD', 'legacySD']:
        image_to_use = alData.img if params['threshUnits'] == 'SD' else alData.smooth
        threshInt = np.mean(image_to_use[image_to_use > 0]) + params['threshLevel'] * np.std(image_to_use[image_to_use > 0])
    else: raise ValueError("Unsupported threshUnits")
    if verbose: print(f"Threshold value is {threshInt} in absolute units")

    # Find local maxima
    minDist = round(params['minDistBetweenSpots'])
    if minDist > 0:
        struct_size = [2 * minDist + 1] * params['numdim'] # create the structural element with the desired size
        struct = np.ones(struct_size, dtype=bool)
        center = tuple([minDist] * params['numdim'])
        struct[center] = False  # Setting the center to False
        
        # image_to_process = alData.img
        image_to_process = alData.smooth
        local_max = image_to_process >= ndimage.grey_dilation(image_to_process, footprint=struct)
    else:
        local_max = np.ones_like(image_to_process, dtype=bool)

    # Enforce that local maxima intensity is above threshold
    local_max &= image_to_process > threshInt

    # Store maxima as a list of coordinates / intensity
    indices = np.argwhere(local_max)
    intensities = image_to_process[local_max]
    maxima = np.column_stack((indices, intensities))
    
    # Ordering the maxima by descending intensity value
    maxima = maxima[maxima[:, -1].argsort()[::-1]]

    # Truncating the array if it has more spots than allowed
    if maxima.shape[0] > params['maxSpots']:
        maxima = maxima[:params['maxSpots'], :]
        if verbose: print(f"Predetected {maxima.shape[0]} spots; Truncated to {params['maxSpots']} spots;")
    elif verbose: print(f"Predetected {maxima.shape[0]} spots;")

    return maxima

## background correction

In [ ]:
import numpy as np

def compute_ROI_boundaries(stack_size, spot_ctr, cutwidth, thickness, ROI_size):
    """
    Compute the boundaries of an ROI around a spot center in an image stack.

    Parameters:
    - stack_size: Tuple of integers representing the size of the stack (nx, ny, [nz]).
    - spot_ctr: Tuple of floats representing the center of the spot (xc, yc, [zc]).
    - cutwidth: Tuple of integers specifying the cut width in pixels.
    - thickness: Integer specifying the additional thickness for 'large' ROI.
    - ROI_size: String specifying the ROI size ('small' or 'large').

    Returns:
    - new_ctr: Tuple of new center coordinates within the ROI.
    - ROIlimits: Numpy array specifying the min and max bounds of the ROI ([xmin, ymin, (zmin)], [xmax, ymax, (zmax)]).
    """

    # Check input consistency
    # print('spot_ctr:', spot_ctr)
    if len(stack_size) == 3 and len(spot_ctr) < 3:
        print('Cannot compute ROI boundaries, incorrect ROI center coordinates.')
        return None, None

    # Compute half length of ROI square/cube
    if ROI_size == 'small':
        halflength = [cutwidth[0]]
    elif ROI_size == 'large':
        halflength = [cutwidth[0] + thickness]

    if len(stack_size) == 3:
        halflength.append(cutwidth[1] if ROI_size == 'small' else cutwidth[1] + thickness)

    # Compute ROI limits
    nx, ny = stack_size[:2]
    xc, yc = spot_ctr[:2]
    xmin, xmax = np.clip([np.ceil(xc - halflength[0]), np.ceil(xc + halflength[0])], 0, nx-1)
    ymin, ymax = np.clip([np.ceil(yc - halflength[0]), np.ceil(yc + halflength[0])], 0, ny-1)
    ROIlimits = np.array([[xmin, ymin], [xmax, ymax]])

    # Adjust for 3D
    if len(stack_size) == 3:
        nz = stack_size[2]
        zc = spot_ctr[2]
        zmin, zmax = np.clip([np.round(zc - halflength[1]), np.round(zc + halflength[1])], 0, nz-1)
        ROIlimits = np.hstack((ROIlimits, [[zmin], [zmax]]))

    # Compute coordinates of spot center in new region
    new_ctr = (xc - xmin + 1, yc - ymin + 1) + ((zc - zmin + 1,) if len(stack_size) == 3 else ())
    # print('ROIlimits_in:', ROIlimits.astype(int))
    return new_ctr, ROIlimits

In [ ]:
from tifffile import imsave
import numpy as np
import os

def generate_bg_region_3D(alData, xcenter, ycenter, zcenter, cutwidth_xy, cutwidth_z, thickness):
    """
    Generate the background region for a 3D dataset.
    
    Parameters:
    - alData: A data structure or array containing the image data.
    - xcenter, ycenter, zcenter: Center coordinates of the spot.
    - cutwidth_xy, cutwidth_z: Cutwidth parameters for defining the ROI.
    - thickness: Defines the thickness of the region around the ROI used for background calculation.
    
    Returns:
    - bg: An array of points in the format [x, y, z, intensity], where x, y, z
      are pixel positions and intensity is the value from the original image.
    """
    # Ensuring integer values for calculations
    thickness = int(abs(thickness))
    cutwidth_xy = int(abs(cutwidth_xy))
    cutwidth_z = int(abs(cutwidth_z))
    nx, ny, nz = alData.img.shape

    # Define the ROI based on provided center and cutwidth
    xc, yc, zc = np.ceil([xcenter, ycenter, zcenter]).astype(int)
    xc = np.clip(xc, 1, nx)
    yc = np.clip(yc, 1, ny)
    zc = np.clip(zc, 1, nz)
    
    # Adjusting ranges to Python's 0-based indexing by subtracting 1
    x2, y2, z2 = np.clip([xc - cutwidth_xy, yc - cutwidth_xy, zc - cutwidth_z], 0, [nx, ny, nz])
    x3, y3, z3 = np.clip([xc + cutwidth_xy, yc + cutwidth_xy, zc + cutwidth_z], 0, [nx, ny, nz])
    
    # Expand the ROI by 'thickness' pixels to define the region for background calculation
    x1, y1, z1 = np.clip([x2 - thickness, y2 - thickness, z2 - thickness], 0, [nx, ny, nz])
    x4, y4, z4 = np.clip([x3 + thickness, y3 + thickness, z3 + thickness], 0, [nx, ny, nz])
    
    # Initialize the background array
    bg = []

    # Loop over the expanded ROI and collect background pixels
    # Including logic to select points within 'thickness' of the ROI
    # Similar to the MATLAB loops for collecting bg points
    for zpix in range(z1, z4):
        for ypix in range(y1, y4):
            for xpix in range(x1, x4):
                # Check if the current pixel is within the 'thickness' boundary of the ROI
                if (xpix < x2 or xpix >= x3) or (ypix < y2 or ypix >= y3) or (zpix < z2 or zpix >= z3):
                    bg.append([xpix + 0.5, ypix + 0.5, zpix, alData.img[xpix, ypix, zpix]])

    bg = np.array(bg)  # Convert list to numpy array for easier handling
    # print(bg.shape, bg.size)
    return bg


def generate_bg_region_2D(alData, xcenter, ycenter, cutwidth_xy, thickness):
    # Adjust for Python's zero-based indexing
    thickness = int(abs(thickness))
    cutwidth_xy = int(abs(cutwidth_xy))
    
    # Determine if alData is handling movie data and adjust accordingly
    img = alData.img
    
    nx, ny = img.shape[:2]
    npts = 2 * thickness * (2 * (thickness + cutwidth_xy) + 1)
    bg = np.zeros((npts+1, 3))
    
    # Adjust for Python's zero-based indexing
    xc = int(np.ceil(xcenter) - 1)
    yc = int(np.ceil(ycenter) - 1)
    xc = max(0, min(xc, nx-1))
    yc = max(0, min(yc, ny-1))
    
    x2, y2 = max(xc - cutwidth_xy, 0), max(yc - cutwidth_xy, 0)
    x3, y3 = min(xc + cutwidth_xy, nx-1), min(yc + cutwidth_xy, ny-1)
    
    x1, y1 = max(xc - cutwidth_xy - thickness, 0), max(yc - cutwidth_xy - thickness, 0)
    x4, y4 = min(xc + cutwidth_xy + thickness, nx-1), min(yc + cutwidth_xy + thickness, ny-1)
    
    # Collect background points
    k = 0
    for xpix in range(x1, x4):
        for ypix in range(y1, y4):
            if (xpix < x2 or xpix > x3) or (ypix < y2 or ypix > y3):
                bg[k, 0] = xpix + 0.5  # Adjusting index for human-readable format (origin at 1,1)
                bg[k, 1] = ypix + 0.5
                bg[k, 2] = img[xpix, ypix]
                k += 1

    bg = bg[:k, :]  # Trim the unused part of the array
    
    # Adjust for spatial coordinates (origin at 0,0)
    bg[:, 0] -= 0.5
    bg[:, 1] -= 0.5
    
    return bg



import numpy as np

def fit_to_3D_plane(data):
    """
    Fits the intensity I vs. (x,y) to a 3D-hyperplane with equation I = ax + by + c.
    Data should be formatted as 3 columns: x, y, intensity, where x, y refer to the
    physical coordinates (in pixel units) of the center of the pixel (origin at 0,0
    at the corner of the image).
    
    Parameters:
    - data: A numpy array of shape (n, 3) where n is the number of points.
    
    Returns:
    - x: A numpy array containing the coefficients [a, b, c] of the plane.
    """
    # Calculate sums required for the matrix equation
    sx = np.sum(data[:, 0])
    sy = np.sum(data[:, 1])
    sI = np.sum(data[:, 2])
    sxx = np.sum(data[:, 0]**2)
    syy = np.sum(data[:, 1]**2)
    sxy = np.sum(data[:, 0]*data[:, 1])
    sIx = np.sum(data[:, 2]*data[:, 0])
    sIy = np.sum(data[:, 2]*data[:, 1])
    npts = data.shape[0]

    # Construct the matrix equation Ax = v for the least squares solution
    fitmat = np.array([[sxx, sxy, sx], [sxy, syy, sy], [sx, sy, npts]])
    v = np.array([sIx, sIy, sI])

    # Solve the matrix equation, handling singular matrix case
    try:
        x = np.linalg.solve(fitmat, v)
    except np.linalg.LinAlgError:  # If matrix inversion is impossible
        x = np.array([0, 0, np.mean(data[:, 2])])

    return x


def fit_to_4D_plane(data):
    """
    Fits the intensity I vs. (x,y,z) to a 4D hyperplane with the equation I = ax + by + cz + d.
    Data should be formatted as 4 columns: x, y, z, intensity, where x, y, z refer to the
    physical coordinates (in pixel units) of the center of the pixel (origin at 0,0,0).
    
    Parameters:
    - data: A numpy array of shape (n, 4) where n is the number of points.
    
    Returns:
    - x: A numpy array containing the coefficients [a, b, c, d] of the plane.
    """
    # Ensure data is in double precision
    data = np.array(data, dtype=float)

    # Summations required for the matrix equation Ax = v
    sx = np.sum(data[:, 0])
    sy = np.sum(data[:, 1])
    sz = np.sum(data[:, 2])
    sI = np.sum(data[:, 3])
    
    # Summations for squared terms
    sxx = np.sum(data[:, 0]**2)
    syy = np.sum(data[:, 1]**2)
    szz = np.sum(data[:, 2]**2)
    
    # Summations for product terms
    sxy = np.sum(data[:, 0] * data[:, 1])
    sxz = np.sum(data[:, 0] * data[:, 2])
    syz = np.sum(data[:, 1] * data[:, 2])
    
    # Summations for product of intensity and coordinates
    sIx = np.sum(data[:, 3] * data[:, 0])
    sIy = np.sum(data[:, 3] * data[:, 1])
    sIz = np.sum(data[:, 3] * data[:, 2])
    
    # Matrix and vector for the linear system
    fitmat = np.array([
        [sxx, sxy, sxz, sx],
        [sxy, syy, syz, sy],
        [sxz, syz, szz, sz],
        [sx, sy, sz, data.shape[0]]
    ])
    v = np.array([sIx, sIy, sIz, sI])

    # Solve the system, handling the case of a singular matrix
    try:
        x = np.linalg.solve(fitmat, v)
    except np.linalg.LinAlgError:  # If matrix inversion is impossible
        x = np.array([0, 0, 0, np.mean(data[:, 3])])

    return x


def final_3D_plane_small(alData, xc, yc, cutwidth_xy, thickness, plfit, ROIsize):
    """
    Generates a 2D image (pl) where pixel values within a specified ROI around
    (xc, yc) are set to values computed by planar interpolation of the background.
    
    Parameters:
    - alData: An object or structure with .img attribute or key containing the image data.
              It can be a 2D array or a 3D array for movie data, with an additional .curFrame attribute or key.
    - xc, yc: Center coordinates of the ROI in the original image.
    - cutwidth_xy: Defines the half-size of the ROI around the center point.
    - thickness: Specifies the thickness for ROI boundary calculation.
    - plfit: Coefficients [a, b, c] of the plane fitting the background.
    - ROIsize: 'small' or 'large', affects the size of the computed ROI.

    Returns:
    - pl: Interpolated plane values within the ROI.
    - stack_bg_corr: The original image corrected by subtracting the interpolated plane.
    - new_ctr: New center of the ROI in the corrected image (adjusted for possible image boundary effects).
    - ROIlimits: The boundaries of the computed ROI.
    """
    if hasattr(alData, 'isMovie') and alData.isMovie:
        img = alData.img[:, :, alData.curFrame]
    else:
        img = alData.img
    
    new_ctr, ROIlimits = compute_ROI_boundaries(img.shape[:2], [xc, yc], cutwidth_xy, thickness, ROIsize)
    
    xmin, xmax = ROIlimits[0, 0], ROIlimits[1, 0]
    ymin, ymax = ROIlimits[0, 1], ROIlimits[1, 1]
    
    pl = np.zeros((xmax-xmin+1, ymax-ymin+1))
    stack_bg_corr = np.zeros_like(pl)
    
    for xpix in range(xmin, xmax+1):
        for ypix in range(ymin, ymax+1):
            interpolated_value = plfit[0]*(xpix-0.5) + plfit[1]*(ypix-0.5) + plfit[2]
            pl[xpix-xmin, ypix-ymin] = interpolated_value
            stack_bg_corr[xpix-xmin, ypix-ymin] = img[xpix, ypix] - interpolated_value
    
    return pl, stack_bg_corr, new_ctr, ROIlimits


def final_4D_plane_small(alData, xc, yc, zc, cutwidth_xy, cutwidth_z, thickness, plfit, ROIsize):
    """
    Generates an image pl within an ROI defined around a central spot (xc, yc, zc)
    in a 3D image. Pixels within the ROI are set to the values computed by a planar
    interpolation of the background surrounding the ROI.
    
    Parameters:
    - alData: Object or dictionary with .img attribute as a 3D numpy array representing the original image data.
    - xc, yc, zc: Coordinates of the central spot.
    - cutwidth_xy, cutwidth_z: Cutwidth parameters defining the size of the ROI.
    - thickness: Not directly used in this function but passed to compute_ROI_boundaries.
    - plfit: Coefficients [a, b, c, d] of the plane fitting the background.
    - ROIsize: Option ('small' or 'large') that affects ROI boundaries computation.
    
    Returns:
    - pl: Interpolated plane values within the ROI.
    - stack_bg_corr: The original image corrected by subtracting the interpolated plane.
    - new_ctr: New center of the ROI in the corrected image.
    - ROIlimits: Boundaries of the ROI.
    """
    new_ctr, ROIlimits = compute_ROI_boundaries(alData.img.shape, [xc, yc, zc], [cutwidth_xy, cutwidth_z], thickness, ROIsize)

    xmin, xmax = int(ROIlimits[0, 0]), int(ROIlimits[1, 0])
    ymin, ymax = int(ROIlimits[0, 1]), int(ROIlimits[1, 1])
    zmin, zmax = int(ROIlimits[0, 2]), int(ROIlimits[1, 2])

    # Initialize the output arrays
    # print(xmax, xmin, ymax, ymin, zmax, zmin)
    pl = np.zeros((xmax-xmin, ymax-ymin, zmax-zmin))
    stack_bg_corr = np.zeros_like(pl)
    

    xgrid, ygrid, zgrid = np.meshgrid(range(xmin, xmax), range(ymin, ymax), range(zmin, zmax), indexing='ij')

    # Calculate interpolated values using vectorized operations
    interpolated_values = plfit[0] * (xgrid+0.5) + plfit[1] * (ygrid+0.5) + plfit[2] * zgrid + plfit[3]
    # Update 'pl' and 'stack_bg_corr' arrays
    pl[:] = interpolated_values
    # print(xmin, xmax)
    # print(alData.img[xmin:xmax, ymin:ymax, zmin:zmax].shape, interpolated_values.shape, stack_bg_corr.shape)
    stack_bg_corr[:] = alData.img[xmin:xmax, ymin:ymax, zmin:zmax] - interpolated_values

    # Compute the interpolated plane and corrected image
    # for xpix in range(xmin, xmax):
    #     for ypix in range(ymin, ymax):
    #         for zpix in range(zmin, zmax):
    #             interpolated_value = (plfit[0] * (xpix-0.5) + plfit[1] * (ypix-0.5) + plfit[2] * zpix + plfit[3])
    #             pl[xpix-xmin, ypix-ymin, zpix-zmin] = interpolated_value
    #             stack_bg_corr[xpix-xmin, ypix-ymin, zpix-zmin] = (alData.img[xpix, ypix, zpix] - interpolated_value)

    return pl, stack_bg_corr, new_ctr, ROIlimits


def gen_linear_interpol(alData, spotCenter, cutwidth, thickness, ROIsize='small'):
    numDim = 3 if len(alData.img.shape) == 3 else 2
    cutwidth_xy = cutwidth[0]
    xc, yc = spotCenter[:2]

    if numDim == 3:
        if len(cutwidth) < 2:
            print("3D stack selected but cutwidth parameter has only 1 element; ensure that fit entry is compatible with 3D data.")
            return [np.array([])] * 7  # Return empty arrays for all outputs
        cutwidth_xy, cutwidth_z = cutwidth[:2]
        
        if len(spotCenter) < 3:
            print("3D stack selected but spot center has only 2 coordinates; ensure that numdim is set to 3.")
            return [np.array([])] * 7  # Return empty arrays for all outputs
        
        zc = spotCenter[2]
        bg = generate_bg_region_3D(alData, spotCenter[0], spotCenter[1], zc, cutwidth_xy, cutwidth_z, thickness=thickness)
    else:
        # Assuming cutwidth and spotCenter are correctly formatted for 2D
        bg = generate_bg_region_2D(alData, spotCenter[0], spotCenter[1], cutwidth[0], thickness)

    if bg.size == 0:
        if numDim == 3:
            plfit = np.zeros(4)  # For 3D data, plfit has 4 elements
            new_ctr = np.array([xc, yc, zc])
            ROIlimits = np.array([1, 1, 1])
        else:
            plfit = np.zeros(3)  # For 2D data, plfit has 3 elements
            new_ctr = np.array([xc, yc])
            ROIlimits = np.array([1, 1])
        
        pl = np.array([])  # Initialize as an empty array
        stack_bg_corr = np.array([])  # Initialize as an empty array
        plmean = 0
        return pl, stack_bg_corr, new_ctr, ROIlimits, plmean, plfit, bg
    

    # Fit to a plane based on dimensionality
    if numDim == 3: plfit = fit_to_4D_plane(bg)  # bg should be prepared beforehand
    else: plfit = fit_to_3D_plane(bg)  # Implement these functions in Python

    # Generate corrected images and ROI based on the fitting
    if numDim == 3: pl, stack_bg_corr, new_ctr, ROIlimits = final_4D_plane_small(alData, spotCenter[0], spotCenter[1], spotCenter[2], cutwidth[0], cutwidth[1], thickness, plfit, ROIsize)
    else: pl, stack_bg_corr, new_ctr, ROIlimits = final_3D_plane_small(alData, spotCenter[0], spotCenter[1], cutwidth[0], thickness, plfit, ROIsize)

    # Calculate mean intensity in the ROI
    plmean = np.mean(pl)
    return pl, stack_bg_corr, new_ctr, ROIlimits, plmean, plfit, bg


def subtract_median(alData, spotCenter, cutWidth, thickness, ROIsize, median_type):
    """
    Subtracts the median (local or global) from a specified ROI in an image or movie frame.
    
    Parameters:
    - alData: An object with .img attribute containing the image data and optionally .isMovie and .curFrame for movie data.
    - spotCenter: The center of the ROI (x, y, [z]).
    - cutWidth: Defines the size of the ROI around the center.
    - thickness: Used in calculating ROI boundaries.
    - ROIsize: 'small' or 'large', affects the size of the computed ROI.
    - median_type: 'local' or 'global', determines the median calculation method.
    
    Returns:
    - pl: An empty array (placeholder for compatibility with other functions).
    - stack_bg_corr: The ROI from the image with the median subtracted.
    - new_ctr: New center of the ROI (adjusted for possible image boundary effects).
    - ROIlimits: The boundaries of the computed ROI.
    """
    img = alData.img

    new_ctr, ROIlimits = compute_ROI_boundaries(img.shape, spotCenter, cutWidth, thickness, ROIsize)

    # Extract the ROI from the image
    stack_bg_corr = img[
        ROIlimits[0, 0]:ROIlimits[1, 0] + 1,
        ROIlimits[0, 1]:ROIlimits[1, 1] + 1,
    ]

    # Subtract the median
    if median_type == 'local':
        median_value = np.median(stack_bg_corr)
    elif median_type == 'global':
        median_value = np.median(img)
    
    stack_bg_corr = stack_bg_corr - median_value

    pl = np.array([])  # Placeholder, not used in this function
    return pl, stack_bg_corr, new_ctr, ROIlimits


# gaussian_mask

In [ ]:
import numpy as np
from scipy.special import erf


def intensity_gaussian3D(xp, yp, zp, x, y, z, sxy, sz, dx, dy, dz):
    """
    Computes the intensity of pixels assuming a standard 3D Gaussian PSF.

    Parameters:
    - xp, yp, zp: arrays of pixel indices.
    - x, y, z: position of the Gaussian center in real units (use dx=dy=dz=1 for pixel units).
    - sxy: sigma in the xy-plane in pixels.
    - sz: sigma in the z-direction in pixels.
    - dx, dy, dz: voxel dimensions in real units.

    Returns:
    - intensity: a 3D array of the Gaussian intensity values.
    """
    # Computing the center position of each voxel
    xp_center = np.ceil(xp) - 0.5
    yp_center = np.ceil(yp) - 0.5
    zp_center = np.round(zp)
    
    # Gaussian intensity calculations
    gx = np.exp(-((xp_center * dx - x) ** 2) / (2 * sxy ** 2))
    gy = np.exp(-((yp_center * dy - y) ** 2) / (2 * sxy ** 2))
    gz = np.exp(-((zp_center * dz - z) ** 2) / (2 * sz ** 2))
    
    intensity = gx * gy * gz
    return intensity


def intensity_integrated_gaussian3D(xp, yp, zp, x, y, z, sxy, sz, dx, dy, dz):
    """
    Computes the intensity of pixels assuming a 3D Gaussian PSF that is
    integrated over each voxel (normalized intensity, i.e., no prefactor).

    Parameters:
    - xp, yp, zp: arrays of pixel indices.
    - x, y, z: position of the Gaussian in real units (if pixels, enter dx=dy=dz=1).
    - sxy: sigma in the xy-plane in pixels.
    - sz: sigma in the z-direction in pixels.
    - dx, dy, dz: voxel dimensions in real units.
    
    Returns:
    - intensity: a 3D array of the Gaussian intensity values integrated over each voxel.
    """
    # Compute the center position of each voxel
    xp_center = np.ceil(xp) - 0.5
    yp_center = np.ceil(yp) - 0.5
    zp_center = np.round(zp)
    
    # Compute the differences for integration limits
    diffx1 = (xp_center - 0.5) * dx - x
    diffx2 = (xp_center + 0.5) * dx - x
    diffy1 = (yp_center - 0.5) * dy - y
    diffy2 = (yp_center + 0.5) * dy - y
    diffz1 = (zp_center - 0.5) * dz - z
    diffz2 = (zp_center + 0.5) * dz - z
    
    # Normalize differences by the standard deviations
    diffx1 /= np.sqrt(2) * sxy
    diffx2 /= np.sqrt(2) * sxy
    diffy1 /= np.sqrt(2) * sxy
    diffy2 /= np.sqrt(2) * sxy
    diffz1 /= np.sqrt(2) * sz
    diffz2 /= np.sqrt(2) * sz
    
    # Calculate the integrated intensity
    intensity = np.abs(erf(diffx1) - erf(diffx2)) * \
                np.abs(erf(diffy1) - erf(diffy2)) * \
                np.abs(erf(diffz1) - erf(diffz2))
    # print(diffx1.shape, diffx2.shape, 
    #       diffy1.shape, diffy2.shape, 
    #       diffz1.shape, diffz2.shape, 
    #       intensity.shape)
    return intensity


def intensity_integrated_gaussian3D_stdZ(xp, yp, zp, x, y, z, sxy, sz, dx, dy, dz):
    """
    Computes intensity of pixels assuming a 3D Gaussian PSF that is
    integrated over each pixel in the xy-plane; along z, the envelope is that of a standard
    Gaussian (no integration).
    
    Parameters:
    - xp, yp, zp: arrays of pixel indices.
    - x, y, z: position of the Gaussian in real units (if pixels, enter dx=dy=dz=1).
    - sxy: sigma in the xy-plane in pixels.
    - sz: sigma in the z direction in pixels.
    - dx, dy, dz: voxel dimensions in real units.
    
    Returns:
    - intensity: a 3D array of the Gaussian intensity values.
    """
    # Computing the center position of each voxel
    xp_center = np.ceil(xp) - 0.5
    yp_center = np.ceil(yp) - 0.5
    zp_center = np.round(zp)
    
    # Calculate differences for erf function in xy plane
    diffx1 = (xp_center - 0.5) * dx - x
    diffx2 = (xp_center + 0.5) * dx - x
    diffy1 = (yp_center - 0.5) * dy - y
    diffy2 = (yp_center + 0.5) * dy - y
    
    # Normalize differences by the standard deviations
    diffx1 /= np.sqrt(2) * sxy
    diffx2 /= np.sqrt(2) * sxy
    diffy1 /= np.sqrt(2) * sxy
    diffy2 /= np.sqrt(2) * sxy
    
    # Gaussian profile in z without integration
    gz = np.exp(-((zp_center * dz - z) ** 2) / (2 * sz ** 2))
    
    # Calculate integrated intensity
    intensity = np.abs(erf(diffx1) - erf(diffx2)) * \
                np.abs(erf(diffy1) - erf(diffy2)) * \
                gz
    return intensity


def intensity_gaussian2D(xp, yp, x, y, sxy, dx, dy):
    """
    Computes the intensity of pixels assuming a standard 2D Gaussian PSF.
    
    Parameters:
    - xp, yp: Arrays of pixel indices.
    - x, y: Position of the Gaussian center in real units (use dx=1, dy=1 for pixel units).
    - sxy: Sigma for the Gaussian in pixels.
    - dx, dy: Pixel dimensions in real units.
    
    Returns:
    - intensity: A 2D array of the Gaussian intensity values.
    """
    # Compute the center position of each pixel
    xp_center = np.ceil(xp) - 0.5
    yp_center = np.ceil(yp) - 0.5
    
    # Calculate the Gaussian intensity
    gx = np.exp(-((xp_center * dx - x) ** 2) / (2 * sxy ** 2))
    gy = np.exp(-((yp_center * dy - y) ** 2) / (2 * sxy ** 2))
    intensity = gx * gy
    
    return intensity


def intensity_integrated_gaussian2D(xp, yp, x, y, sxy, dx, dy):
    """
    Computes the intensity of pixels assuming an integrated 2D Gaussian PSF.
    
    Parameters:
    - xp, yp: Arrays of pixel indices.
    - x, y: Position of the Gaussian center in real units (use dx=dy=1 for pixel units).
    - sxy: Sigma for the Gaussian in pixels.
    - dx, dy: Pixel dimensions in real units.
    
    Returns:
    - intensity: A 2D array of the integrated Gaussian intensity values.
    """
    # Adjust xp and yp to the center position of each pixel
    xp_center = np.ceil(xp) - 0.5
    yp_center = np.ceil(yp) - 0.5
    
    # Compute differences for the erf function
    diffx1 = (xp_center - 0.5) * dx - x
    diffx2 = (xp_center + 0.5) * dx - x
    diffy1 = (yp_center - 0.5) * dy - y
    diffy2 = (yp_center + 0.5) * dy - y
    
    # Normalize differences by the standard deviations
    diffx1 /= np.sqrt(2) * sxy
    diffx2 /= np.sqrt(2) * sxy
    diffy1 /= np.sqrt(2) * sxy
    diffy2 /= np.sqrt(2) * sxy
    
    # Calculate the integrated intensity using the error function
    intensity = np.abs(erf(diffx1) - erf(diffx2)) * np.abs(erf(diffy1) - erf(diffy2))
    
    return intensity

In [ ]:
import numpy as np

def gaussian_mask_small(data, spotCenter, params):
    '''
    runs a 2D/3D gaussian mask algorithm to localize and quantify the intensity of a fluorescent spot
    data is the image/stack
    all units in pix with origin at the edge of pixel i.e. leftmost corner
    center corresponds to y = 0.5
    spot_ctr = [x y z] : the guess center coordinates (or [x,y] in 2D)
    params is an airlocalizeParams object holding the fit parameters
        params.psfType : 'gaussian' or 'integratedGaussian' or 'integratedGaussianStdZ' (last option only in 3D)
        params.psfSigma(1) : PSF width (in pixels)
        params.psfSigma(2) : PSF height (in pixels - ignored in 2D)
        params.fittedRegionSize : range (in PSF width units) of the region around the spot used for fitting. 
        params.maxIterations is the maximum number of iterations of the equations allowed before convergence
        params.tol is the tolerance on the convergence (in lateral pixel dimension units)
    '''
    # Convert input parameters
    xs, ys = spotCenter[:2]
    sxy = params['psfSigma'][0]
    tol = params['tol']
    maxcount = params['maxIterations']
    cutSize = params['fittedRegionSize']
    cutwidth = [cutSize * sxy]
    it = 0

    # Initialize arrays
    x0 = np.zeros(maxcount)
    y0 = np.zeros(maxcount)
    z0 = None
    N0 = np.zeros(maxcount)
    dist = np.zeros(maxcount)
    x0[0], y0[0] = xs, ys

    # Handle 3D data
    if data.ndim == 3:
        zs = spotCenter[2]
        sz = params['psfSigma'][1]
        cutwidth.append(cutSize * sz)
        z0 = np.zeros(maxcount)
        z0[0] = zs

    # Compute boundaries of the ROI
    # print(data.shape)
    _, ROIlimits = compute_ROI_boundaries(data.shape, [x0[it], y0[it]] + ([z0[it]] if data.ndim == 3 else []), cutwidth, 0, 'small')
    # Select sub-data within ROI
    if data.ndim == 3:
        # Compute ROI boundaries for 3D data
        xp_min, xp_max = int(ROIlimits[0, 0]), int(ROIlimits[1, 0])
        yp_min, yp_max = int(ROIlimits[0, 1]), int(ROIlimits[1, 1])
        zp_min, zp_max = int(ROIlimits[0, 2]), int(ROIlimits[1, 2])

        # Generating the grid within ROI and selecting sub-data
        xp, yp, zp = np.meshgrid(range(xp_min, xp_max), range(yp_min, yp_max), range(zp_min, zp_max), indexing='ij')
        sdata = data[xp_min:xp_max, yp_min:yp_max, zp_min:zp_max]
    else:
        # Compute ROI boundaries for 2D data
        xp_min, xp_max = int(ROIlimits[0, 0]), int(ROIlimits[1, 0])
        yp_min, yp_max = int(ROIlimits[0, 1]), int(ROIlimits[1, 1])
        # Generating the grid within ROI and selecting sub-data
        xp, yp = np.meshgrid(range(xp_min, xp_max), range(yp_min, yp_max), indexing='ij')
        sdata = data[xp_min:xp_max, yp_min:yp_max]

    # Loop through iterations
    it = 1
    dx, dy, dz = 1, 1, 1
    tol = tol * (dx + dy) / 2.0
    tmp = tol + 1
    while it < maxcount and tmp > tol:
        # Initializations for the iteration
        x, y = x0[it - 1], y0[it - 1]
        if data.ndim == 3: z = z0[it - 1]

        # Placeholder for the intensity calculation based on the PSF type
        # Assume intensity calculation functions are defined elsewhere
        if data.ndim == 3:
            if params['psfType'] == 'gaussian':
                intensity = intensity_gaussian3D(xp, yp, zp, x, y, z, sxy, sz, dx, dy, dz)
            elif params['psfType'] == 'integratedGaussian':
                intensity = intensity_integrated_gaussian3D(xp, yp, zp, x, y, z, sxy, sz, dx, dy, dz)
            elif params['psfType'] == 'integratedGaussianStdZ':
                intensity = intensity_integrated_gaussian3D_stdZ(xp, yp, zp, x, y, z, sxy, sz, dx, dy, dz)
        else:
            if params['psfType'] == 'gaussian':
                intensity = intensity_gaussian2D(xp, yp, x, y, sxy, dx, dy)
            elif params['psfType'] in ['integratedGaussian', 'integratedGaussianStdZ']:
                intensity = intensity_integrated_gaussian2D(xp, yp, x, y, sxy, dx, dy)
        
        intsum = np.sum(intensity * sdata)
        sumsum = np.sum(intensity**2)

        sumx = np.sum((xp - 0.5) * intensity * sdata)
        sumy = np.sum((yp - 0.5) * intensity * sdata)
        if data.ndim == 3: sumz = np.sum(zp * intensity * sdata)

        if intsum <= 0 or sumsum == 0:
            x0[it], y0[it], N0[it] = -1, -1, -1
            if data.ndim == 3: z0[it] = -1
        else:
            x0[it] = sumx / intsum
            y0[it] = sumy / intsum
            if data.ndim == 3: z0[it] = sumz / intsum

            N0[it] = intsum / sumsum

            # Location_out_of_ROI check
            is_outside = False  # Flag to indicate if the location is outside the ROI
            x0_current, y0_current = x0[it], y0[it]
            if x0_current / dx < xp_min - 1 or x0_current / dx >= xp_max: is_outside = True
            if y0_current / dy < yp_min - 1 or y0_current / dy >= yp_max: is_outside = True
            if data.ndim == 3:
                z0_current = z0[it]
                if z0_current / dz < zp_min - 1 or z0_current / dz >= zp_max: is_outside = True

            # If the location is determined to be outside the ROI, update the values to -1
            if is_outside:
                x0[it], y0[it], N0[it] = -1, -1, -1
                if data.ndim == 3: z0[it] = -1

        # Update distance and prepare for the next iteration
        if data.ndim == 3: dist[it] = np.sqrt((x - x0[it])**2 + (y - y0[it])**2 + (z - z0[it])**2)
        else: dist[it] = np.sqrt((x - x0[it])**2 + (y - y0[it])**2)

        tmp = dist[it]
        if x0[it] == -1: tmp = tol - 1  # Force exit if the location is out of ROI

        it += 1


    x0 = x0[it - 1]
    y0 = y0[it - 1]
    if data.ndim == 3: z0 = z0[it - 1]

    N0 = N0[it - 1]
    dist = dist[it - 1]
    
    # Select the relevant sub-array from the data based on the ROI boundaries for error computation
    err0 = np.sqrt(np.sum((N0 * intensity - sdata) ** 2))

    # intensity calculation
    if data.ndim == 3:
        if params['psfType'] == 'integratedGaussian':
            N0 *= 8
        elif params['psfType'] == 'integratedGaussianStdZ':
            # Define a grid around the estimated center
            x = np.arange(np.floor(x0 - 3 * sxy), np.ceil(x0 + 3 * sxy) + 1)
            y = np.arange(np.floor(y0 - 3 * sxy), np.ceil(y0 + 3 * sxy) + 1)
            z = np.arange(np.floor(z0 - 3 * sz), np.ceil(z0 + 3 * sz) + 1)
            xx, yy, zz = np.meshgrid(y, x, z, indexing='ij')
            xx = np.ceil(xx) - 0.5
            yy = np.ceil(yy) - 0.5
            zz = np.round(zz)
            # Calculate the integrated intensity
            Itot = intensity_integrated_gaussian3D_stdZ(xx, yy, zz, x0, y0, z0, sxy, sz, 1, 1, 1)
            N0 *= np.sum(Itot)
        elif params['psfType'] == 'gaussian':
            # Similar grid definition as above
            xx, yy, zz = np.meshgrid(y, x, z, indexing='ij')
            xx = np.ceil(xx) - 0.5
            yy = np.ceil(yy) - 0.5
            zz = np.round(zz)
            # Calculate the Gaussian intensity
            Itot = intensity_gaussian3D(xx, yy, zz, x0, y0, z0, sxy, sz, 1, 1, 1)
            N0 *= np.sum(Itot)
    else:
        if params['psfType'] == 'integratedGaussian':
            N0 *= 4
        elif params['psfType'] == 'integratedGaussianStdZ':
            N0 *= 4  # Assuming this is the same adjustment as for integratedGaussian in 2D
        elif params['psfType'] == 'gaussian':
            # Define a 2D grid around the estimated center
            x = np.arange(np.floor(x0 - 3 * sxy), np.ceil(x0 + 3 * sxy) + 1)
            y = np.arange(np.floor(y0 - 3 * sxy), np.ceil(y0 + 3 * sxy) + 1)
            xx, yy = np.meshgrid(y, x, indexing='ij')
            xx = np.ceil(xx) - 0.5
            yy = np.ceil(yy) - 0.5
            # Calculate the Gaussian intensity
            Itot = intensity_gaussian2D(xx, yy, x0, y0, sxy, 1, 1)
            N0 *= np.sum(Itot)

    return x0, y0, z0, N0, err0, dist, it

# gaussian_fit_local

In [ ]:
import numpy as np


def gauss2D(Coeffs, pts):
    """
    Calculate the 2D Gaussian intensity.

    Args:
    - Coeffs: Coefficients for the Gaussian calculation. An array where:
        - Coeffs[0] is Imax, 
        - Coeffs[1] is background intensity (bg), 
        - Coeffs[2] is standard deviation in x and y (sxy),
        - Coeffs[3] is center in x (xc),
        - Coeffs[4] is center in y (yc).
    - pts: Points where the Gaussian intensity should be calculated. A 2D array 
           where the first dimension is for different points, and the second dimension 
           represents [r, c], corresponding to rows and columns.

    Returns:
    - I: The calculated Gaussian intensity for each point.
    """
    gridSize = pts.shape[1]
    r = pts[:, :gridSize//2]
    c = pts[:, gridSize//2:]

    # Assuming intensity_gaussian2D is defined elsewhere and works similarly to the MATLAB version.
    I = Coeffs[1] + Coeffs[0] * intensity_gaussian2D(r, c, Coeffs[3], Coeffs[4], Coeffs[2], 1, 1)

    return I


def gauss_integrated2D(Coeffs, pts):
    """
    Calculate the integrated 2D Gaussian intensity.

    Args:
    - Coeffs: Coefficients for the Gaussian calculation. An array where:
        - Coeffs[0] is Imax,
        - Coeffs[1] is background intensity (bg),
        - Coeffs[2] is standard deviation in x and y (sxy),
        - Coeffs[3] is center in x (xc),
        - Coeffs[4] is center in y (yc).
    - pts: Points where the Gaussian intensity should be calculated. A 2D array
           where the first dimension is for different points, and the second dimension
           represents [r, c], corresponding to rows and columns.

    Returns:
    - I: The calculated integrated Gaussian intensity for each point.
    """
    gridSize = pts.shape[1]
    r = pts[:, :gridSize//2]
    c = pts[:, gridSize//2:]

    # Compute max intensity so prefactor (Coeffs[0]) is actually Imax.
    I0 = intensity_integrated_gaussian2D(1, 1, 0.5, 0.5, Coeffs[2], 1, 1)

    # Compute intensity over all data points.
    I = Coeffs[1] + Coeffs[0] * intensity_integrated_gaussian2D(
        r, c, Coeffs[3], Coeffs[4], Coeffs[2], 1, 1) / I0

    return I


def gauss3D(Coeffs, pts):
    """
    Calculate the 3D Gaussian intensity.

    Args:
    - Coeffs: Coefficients for the Gaussian calculation. An array where:
        - Coeffs[0] is Imax, 
        - Coeffs[1] is background intensity (bg), 
        - Coeffs[2] is standard deviation in x and y (sxy),
        - Coeffs[3] is standard deviation in z (sz),
        - Coeffs[4] is center in x (xc),
        - Coeffs[5] is center in y (yc),
        - Coeffs[6] is center in z (zc).
    - pts: Points where the Gaussian intensity should be calculated. A 3D array 
           where the first dimension is for different points, and the second dimension 
           represents [r, c, h], corresponding to rows, columns, and height.

    Returns:
    - I: The calculated Gaussian intensity for each point.
    """
    gridSize = pts.shape[1]
    r = pts[:, :gridSize//3]
    c = pts[:, gridSize//3:2*gridSize//3]
    h = pts[:, 2*gridSize//3:]

    # Assuming intensity_gaussian3D is defined elsewhere and works similarly to the MATLAB version.
    I = Coeffs[1] + Coeffs[0] * intensity_gaussian3D(r, c, h, Coeffs[4], Coeffs[5], Coeffs[6], Coeffs[2], Coeffs[3], 1, 1, 1)

    return I


def gauss_integrated3D(Coeffs, pts):
    """
    Calculate the integrated 3D Gaussian intensity.

    Args:
    - Coeffs: Coefficients for the Gaussian calculation. An array where:
        - Coeffs[0] is Imax, 
        - Coeffs[1] is background intensity (bg), 
        - Coeffs[2] is standard deviation in x and y (sxy),
        - Coeffs[3] is standard deviation in z (sz),
        - Coeffs[4] is center in x (xc),
        - Coeffs[5] is center in y (yc),
        - Coeffs[6] is center in z (zc).
    - pts: Points where the Gaussian intensity should be calculated. A 3D array 
           where the first dimension is for different points, and the second dimension 
           represents [r, c, h], corresponding to rows, columns, and planes.

    Returns:
    - I: The calculated integrated Gaussian intensity for each point.
    """
    gridSize = pts.shape[1]
    r = pts[:, :gridSize//3]
    c = pts[:, gridSize//3:2*gridSize//3]
    h = pts[:, 2*gridSize//3:]

    # Compute max intensity so prefactor (Coeffs[1]) is actually Imax.
    # Assuming intensity_integrated_gaussian3D is defined elsewhere and correctly adapted from MATLAB.
    I0 = intensity_integrated_gaussian3D(1, 1, 1, 0.5, 0.5, 1, Coeffs[2], Coeffs[3], 1, 1, 1)

    # Compute intensity over all data points.
    I = Coeffs[1] + Coeffs[0] * intensity_integrated_gaussian3D(
        r, c, h, Coeffs[4], Coeffs[5], Coeffs[6], Coeffs[2], Coeffs[3], 1, 1, 1) / I0

    return I


def gauss_integrated3D_stdZ(Coeffs, pts):
    """
    Calculate the integrated 3D Gaussian intensity with standardized z-dimension.

    Args:
    - Coeffs: Coefficients for the Gaussian calculation. An array where:
        - Coeffs[0] is Imax,
        - Coeffs[1] is background intensity (bg),
        - Coeffs[2] is standard deviation in x and y (sxy),
        - Coeffs[3] is standard deviation in z (sz),
        - Coeffs[4] is center in x (xc),
        - Coeffs[5] is center in y (yc),
        - Coeffs[6] is center in z (zc).
    - pts: Points where the Gaussian intensity should be calculated. A 3D array
           where the first dimension is for different points, and the second dimension
           represents [r, c, h], corresponding to rows, columns, and planes.

    Returns:
    - I: The calculated integrated Gaussian intensity for each point.
    """
    gridSize = pts.shape[1]
    r = pts[:, :gridSize//3]
    c = pts[:, gridSize//3:2*gridSize//3]
    h = pts[:, 2*gridSize//3:]

    # Compute max intensity so prefactor (Coeffs[0]) is actually Imax.
    I0 = intensity_integrated_gaussian3D_stdZ(
        1, 1, 1, 0.5, 0.5, 1, Coeffs[2], Coeffs[3], 1, 1, 1)

    # Compute intensity over all data points.
    I = Coeffs[1] + Coeffs[0] * intensity_integrated_gaussian3D_stdZ(
        r, c, h, Coeffs[4], Coeffs[5], Coeffs[6], Coeffs[2], Coeffs[3], 1, 1, 1) / I0

    return I

In [ ]:
import numpy as np
from scipy.optimize import curve_fit


def gaussian_fit_local(alData, spotCenter, params, background_correction):
    """
    Performs Gaussian fitting on local image data.

    Parameters:
    - alData: Contains the image data and movie flag.
    - spotCenter: Guess center coordinates [x, y] or [x, y, z].
    - params: Fit parameters.
    - background_correction: Flag for background correction (1 or 0).

    Returns:
    - Gaussout: Array with the fitting results.
    """
    dx = dy = 1  # Assuming unity voxel dimensions for simplicity
    numDim = alData['img'].ndim
    xc, yc = spotCenter[0], spotCenter[1]
    zc = spotCenter[2] if numDim == 3 else None

    if alData['img'].ndim == 3: nx, ny, nz = alData['img'].shape
    else: nx, ny = alData['img'].shape[:2]  # Only extract the first two dimensions


    # Set default or use provided parameters
    sxy = params.get('psfSigma', [2.0])[0] / dx
    sz = params.get('psfSigma', [2.0, 2.0])[1] / dy if numDim == 3 else 2.0
    cutSize = params.get('fittedRegionSize', 3)
    tol = params.get('tol', 1e-6)
    maxcount = params.get('maxIterations', 200)
    psfType = params.get('psfType', 'integratedGaussianStdZ' if numDim == 3 else 'integratedGaussian')
    
    # Generate data and perform background correction
    _, data, new_ctr, ROIlimits, _, plfit, _ = gen_linear_interpol(alData, spotCenter, [cutSize * sxy, cutSize * sz] if numDim == 3 else cutSize * sxy, 1, 'large')

    if background_correction:
        if numDim == 2:
            xmin, ymin = ROIlimits[0, 0], ROIlimits[0, 1]
            xc2, yc2 = new_ctr[0], new_ctr[1]

            nrows, ncols = data.shape[:2]
            r, c = np.meshgrid(np.arange(1, ncols+1), np.arange(1, nrows+1), indexing='ij')
            c = np.ceil(c) - 0.5
            r = np.ceil(r) - 0.5
            pts = np.vstack([r.ravel(), c.ravel()]).T  # Transposing to get [r c] pairs
            xmax = xmin + data.shape[0] - 1
            ymax = ymin + data.shape[1] - 1

        elif numDim == 3:
            xmin, ymin, zmin = ROIlimits[0, 0], ROIlimits[0, 1], ROIlimits[0, 2]
            xc2, yc2, zc2 = new_ctr[0], new_ctr[1], new_ctr[2]

            nrows, ncols, nups = data.shape
            r, c, h = np.meshgrid(np.arange(1, ncols+1), np.arange(1, nrows+1), np.arange(1, nups+1), indexing='ij')
            c = np.ceil(c) - 0.5
            r = np.ceil(r) - 0.5
            h = np.round(h)
            pts = np.vstack([r.ravel(), c.ravel(), h.ravel()]).T  # Transposing to get [r c h] triplets
            xmax = xmin + data.shape[0] - 1
            ymax = ymin + data.shape[1] - 1
            zmax = zmin + data.shape[2] - 1
    else:
        npix = [np.ceil(cutSize * sxy), np.ceil(cutSize * sxy), np.ceil(cutSize * sz)] if numDim == 3 and not alData['isMovie'] else [np.ceil(cutSize * sxy), np.ceil(cutSize * sxy)]
        xmin = max(int(np.ceil(xc) - npix[0]), 0)
        xmax = min(int(np.ceil(xc) + npix[0]), nx - 1)  # Python indexing is 0-based
        ymin = max(int(np.ceil(yc) - npix[1]), 0)
        ymax = min(int(np.ceil(yc) + npix[1]), ny - 1)

        if numDim == 3:
            zmin = max(int(np.ceil(zc) - npix[2]), 0)
            zmax = min(int(np.ceil(zc) + npix[2]), nz - 1)
            data = alData['img'][xmin:xmax+1, ymin:ymax+1, zmin:zmax+1]
            r, c, h = np.meshgrid(range(xmin, xmax+1), range(ymin, ymax+1), range(zmin, zmax+1), indexing='ij')
            c = np.ceil(c) - 0.5
            r = np.ceil(r) - 0.5
            h = np.round(h)
            pts = np.vstack([r.ravel(), c.ravel(), h.ravel()]).T
            xc2 = xc - (xmin + 0.5)
            yc2 = yc - (ymin + 0.5)
            zc2 = zc - (zmin + 0.5)
        else:
            data = alData['img'][xmin:xmax+1, ymin:ymax+1]
            r, c = np.meshgrid(range(xmin, xmax+1), range(ymin, ymax+1), indexing='ij')
            pts = np.vstack([r.ravel(), c.ravel()]).T - 0.5
            xc2 = xc - (xmin + 0.5)
            yc2 = yc - (ymin + 0.5)

    # defining guess values of the fit parameters, their acceptable range, and other fit options
    bgmin = np.min(data)
    bg = np.median(data)
    Imax = np.max(data) - bg

    if numDim == 3:
        Coeffs = [Imax, bg, sxy, sz, xc2, yc2, zc2]
        lb = [0, bgmin, 0.1, 0.1, 0, 0, 0]
        ub = [5 * Imax, 0.5 * Imax + bg, (xmax - xmin) / 2, (zmax - zmin) / 2, xmax - xmin + 1, ymax - ymin + 1, zmax - zmin + 1]
    else:
        Coeffs = [Imax, bg, sxy, xc2, yc2]
        lb = [0, bgmin, 0.1, 0, 0]
        ub = [5 * Imax, 0.5 * Imax + bg, (xmax - xmin) / 2, xmax - xmin + 1, ymax - ymin + 1]

    # Setting the appropriate fitting function
    if numDim == 3:
        if psfType == 'gaussian': hfun = gauss3D
        elif psfType == 'integratedGaussian': hfun = gauss_integrated3D
        elif psfType == 'integratedGaussianStdZ': hfun = gauss_integrated3D_stdZ
    elif numDim == 2:
        if psfType == 'gaussian': hfun = gauss2D
        elif psfType == 'integratedGaussian': hfun = gauss_integrated2D
        elif psfType == 'integratedGaussianStdZ': hfun = gauss_integrated2D


    # Actual fit
    Gaussout, cov = curve_fit(hfun, pts, data, p0=Coeffs, bounds=(lb, ub), kwargs={'maxfev': maxcount, 'xtol': tol})
    
    if numDim == 3 and not alData.isMovie:
        if psfType == 'gaussian':
            xc, yc, zc = Gaussout[4], Gaussout[5], Gaussout[6]
            sxy, sz = Gaussout[2], Gaussout[3]
            x = np.arange(np.floor(xc - 3*sxy), np.ceil(xc + 3*sxy) + 1)
            y = np.arange(np.floor(yc - 3*sxy), np.ceil(yc + 3*sxy) + 1)
            z = np.arange(np.floor(zc - 3*sz), np.ceil(zc + 3*sz) + 1)
            xx, yy, zz = np.meshgrid(y, x, z, indexing='ij')
            pts2 = np.stack([np.ceil(xx)-0.5, np.ceil(yy)-0.5, np.round(zz)], axis=-1).reshape(-1, 3)
            bg = Gaussout[1]
            Gaussout[1] = 0
            Itot = np.sum(gauss3D(Gaussout, pts2))
            Gaussout[1] = bg
        elif psfType == 'integratedGaussian':
            I0 = intensity_integrated_gaussian3D(1, 1, 1, 0.5, 0.5, 1, Gaussout[2], Gaussout[3], 1, 1, 1)
            Itot = 8*Gaussout[0]/I0
        elif psfType == 'integratedGaussianStdZ':
            xc, yc, zc = Gaussout[4], Gaussout[5], Gaussout[6]
            sxy, sz = Gaussout[2], Gaussout[3]
            x = np.arange(np.floor(xc - 3*sxy), np.ceil(xc + 3*sxy) + 1)
            y = np.arange(np.floor(yc - 3*sxy), np.ceil(yc + 3*sxy) + 1)
            z = np.arange(np.floor(zc - 3*sz), np.ceil(zc + 3*sz) + 1)
            xx, yy, zz = np.meshgrid(y, x, z, indexing='ij')
            pts2 = np.stack([np.ceil(xx)-0.5, np.ceil(yy)-0.5, np.round(zz)], axis=-1).reshape(-1, 3)
            bg = Gaussout[1]
            Gaussout[1] = 0
            Itot = np.sum(gauss_integrated3D_stdZ(Gaussout, pts2))
            Gaussout[1] = bg

        Gaussout = np.append(Gaussout, [Itot, np.sqrt(cov)])

    elif numDim == 2:
        if psfType == 'gaussian':
            xc, yc, sxy = Gaussout[3], Gaussout[4], Gaussout[2]
            x = np.arange(np.floor(xc - 3*sxy), np.ceil(xc + 3*sxy) + 1)
            y = np.arange(np.floor(yc - 3*sxy), np.ceil(yc + 3*sxy) + 1)
            xx, yy = np.meshgrid(y, x, indexing='ij')
            pts2 = np.stack([np.ceil(xx)-0.5, np.ceil(yy)-0.5], axis=-1).reshape(-1, 2)
            bg = Gaussout[1]
            Gaussout[1] = 0
            Itot = np.sum(gauss2D(Gaussout, pts2))
            Gaussout[1] = bg
        elif psfType == 'integratedGaussian':
            I0 = intensity_integrated_gaussian2D(1, 1, 0.5, 0.5, Gaussout[2], 1, 1)
            Itot = 4*Gaussout[0]/I0

        # Update Gaussout with the total integrated intensity and the square root of the residual norm.
        Gaussout = np.append(Gaussout, [Itot, np.sqrt(cov)])


    # Correcting the center position for the offset of the substack used for the fit
    if numDim == 3:
        Gaussout[4] = Gaussout[4] - 1 + xmin  # Adjust x center
        Gaussout[5] = Gaussout[5] - 1 + ymin  # Adjust y center
        Gaussout[6] = Gaussout[6] - 1 + zmin  # Adjust z center
    else:
        Gaussout[3] = Gaussout[3] - 1 + xmin  # Adjust x center for 2D
        Gaussout[4] = Gaussout[4] - 1 + ymin  # Adjust y center for 2D

    # Adding the extra parameters from the background equation to the output
    if background_correction == 1:
        if numDim == 3: Gaussout = np.append(Gaussout, plfit[:4])  # Assuming plfit has at least 4 elements
        else: Gaussout = np.append(Gaussout, plfit[:3])  # Assuming plfit has at least 3 elements


    return Gaussout

# local_maxproj

In [ ]:
def local_maxproj(alData, spot_ctr, params):
    """
    Compute a localized maximum intensity projection around a specified spot center.

    Args:
    - alData: An object or dictionary containing the imaging data in 'img' key or attribute.
    - spot_ctr: The center of the spot as a tuple or list, [x, y, z].
    - params: A dictionary or object with attributes 'psfSigma' and 'fittedRegionSize', 
              specifying the standard deviation of the PSF and the size of the region to fit, respectively.

    Returns:
    - img: A 2D numpy array representing the localized maximum intensity projection.
    """
    zc = spot_ctr[2]
    cutwidth_z = np.ceil(params['psfSigma'][1] * params['fittedRegionSize'])
    nz = alData['img'].shape[2]
    
    # Calculate the range of planes for the max projection
    ROIlimits_z = np.ceil(zc) - np.floor(cutwidth_z)
    ROIlimits_z = max(1, ROIlimits_z)  # Ensure the lower bound is within the image
    zmax = np.ceil(zc) + np.floor(cutwidth_z)
    zmax = min(nz, zmax)  # Ensure the upper bound is within the image
    
    # Perform the max projection
    img = np.max(alData['img'][:, :, int(ROIlimits_z)-1:int(zmax)], axis=2)  # Adjusted for Python's 0-based indexing

    return img

# run_gaussian_fit_on_all_spots_in_image

In [ ]:
import numpy as np
from tqdm import tqdm


def run_gaussian_fit_on_all_spots_in_image(spotCandidates, alData, params, verbose=True):
    """
    Translated function to run Gaussian fit on all spots in an image.

    Args:
        spotCandidates (np.ndarray): Array of spot candidates.
        alData (dict): Dictionary containing image data and metadata.
        params (dict): Dictionary of parameters for fitting.

    Returns:
        tuple: A tuple containing the location of spots and their variables.
    """
    # Initialize arrays and set cutwidth
    nSpots = len(spotCandidates)
    loc = None
    locVars = None
    cutWidth = None

    if params['fitMethod'] == '3DGaussianFit':
        loc = np.zeros((nSpots, 8))
        locVars = ['x_in_pix', 'y_in_pix', 'z_in_pix', 'integratedIntensity', 'residuals', 'image_number']
        cutWidth = [np.ceil(params['fittedRegionSize'] * psf) for psf in params['psfSigma']]

    elif params['fitMethod'] in ['3DMaskFull', '2DMaskOnLocalMaxProj']:
        loc = np.zeros((nSpots, 6))
        locVars = ['x_in_pix', 'y_in_pix', 'z_in_pix', 'integratedIntensity', 'residuals', 'image_number']
        cutWidth = [np.ceil(params['fittedRegionSize'] * psf) for psf in params['psfSigma']]
        
    elif params['fitMethod'] in ['2DGaussianMask', '2DGaussianFit']:
        loc = np.zeros((nSpots, 4))
        locVars = ['x_in_pix', 'y_in_pix', 'integratedIntensity', 'residuals', 'image_number']
        cutWidth = np.ceil(params['fittedRegionSize'] * params['psfSigma'][0])
    
    else: raise ValueError("Unrecognized fit method")


    # Loop over each pre-detected spot
    with tqdm(total=nSpots, desc=f"Fit predetected spots", position=0, leave=True, disable=not verbose) as pbar_sub:
        for j in range(nSpots):
            new_ctr = spotCandidates[j, :params['numdim']]
            ROIlimits = None
            # Background correction
            if params['bgCorrectionMode'] == 'localPlane':
                _, stack_bg_corr, new_ctr, ROIlimits, *_ = gen_linear_interpol(alData, new_ctr, cutWidth, 1, 'large')
            elif params['bgCorrectionMode'] == 'localMedian':
                stack_bg_corr, new_ctr, ROIlimits = subtract_median(alData, new_ctr, cutWidth, 1, mode='local')
            elif params['bgCorrectionMode'] == 'globalMedian':
                stack_bg_corr, new_ctr, ROIlimits = subtract_median(alData, new_ctr, cutWidth, 1, mode='global')
            
            # Gaussian Fit
            if params['fitMethod'] == '3DMaskFull':
                # print(stack_bg_corr.shape)
                x0, y0, z0, N0, err0, *_ = gaussian_mask_small(stack_bg_corr, new_ctr, params)
                loc[j, 0:5] = [x0 + ROIlimits[0, 0] + 1, y0 + ROIlimits[0, 1] + 1, z0 + ROIlimits[0, 2], N0, err0]
            
            elif params['fitMethod'] == '2DGaussianMask':
                x0, y0, _, N0, err0, *_ = gaussian_mask_small(stack_bg_corr, new_ctr, params)
                loc[j, 0:4] = [x0 + ROIlimits[0, 0], y0 + ROIlimits[0, 1], N0, err0]

            elif params['fitMethod'] == '3DGaussianFit':
                Gaussout = gaussian_fit_local(stack_bg_corr, new_ctr, params, 1)
                loc[j, 0:5] = [Gaussout[4] + ROIlimits[0, 0], Gaussout[5] + ROIlimits[0, 1], Gaussout[6] + ROIlimits[0, 2], Gaussout[7], Gaussout[8]]

            elif params['fitMethod'] == '2DGaussianFit':
                Gaussout = gaussian_fit_local(stack_bg_corr, new_ctr, params, 1)
                loc[j, 0:4] = [Gaussout[3] + ROIlimits[0, 0], Gaussout[4] + ROIlimits[0, 1], Gaussout[5], Gaussout[6]]

            elif params['fitMethod'] == '2DMaskOnLocalMaxProj':
                # Assuming local_maxproj function is defined elsewhere
                img_bg_corr = local_maxproj(stack_bg_corr, new_ctr, params)
                x0, y0, _, N0, err0, *_ = gaussian_mask_small(img_bg_corr, new_ctr[:2], params)
                loc[j, 0:5] = [x0 + ROIlimits[0, 0], y0 + ROIlimits[0, 1], new_ctr[2] + ROIlimits[0, 2], N0, err0]

            else:
                print(f"Unrecognized fit method: {params['fitMethod']}")

            pbar_sub.update(1)


    return loc, locVars

In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from scipy.ndimage import binary_dilation
import tifffile

import numpy as np

def create_spherical_structure(radius):
    """
    Creates a 3D spherical structuring element with the given radius.
    
    Parameters:
    - radius: The radius of the sphere.
    
    Returns:
    - A 3D numpy array with shape (2*radius+1, 2*radius+1, 2*radius+1),
      where elements within the specified radius are True, and others are False.
    """
    # The diameter and size of the structure array
    diameter = radius * 2 + 1
    struct_size = (diameter, diameter, diameter)
    
    # Create an array of distances from the center
    arr = np.zeros(struct_size)
    center = np.array(struct_size) // 2
    for z in range(diameter):
        for y in range(diameter):
            for x in range(diameter):
                distance = np.sqrt((center[0] - z) ** 2 + (center[1] - y) ** 2 + (center[2] - x) ** 2)
                if distance <= radius:
                    arr[z, y, x] = True
                else:
                    arr[z, y, x] = False
    
    return arr.astype(np.bool_)


def save_points(df, shape, output_tiff_path, radius=3, verbose=True):
    # Assuming you have the `shape` of your image from the TIFF metadata
    label_image = np.zeros(shape, dtype=np.uint16)

    # Create the binary image and dilate
    # structure = np.ones((3, 3, 3), dtype=np.bool_)
    structure = create_spherical_structure(radius=radius)

    # Vectorized setting of points in the label_image array
    # Note: Ensure that z, x, y indices are within the bounds of the image shape
    valid_points = (df['z_in_pix'] < shape[0]) & (df['x_in_pix'] < shape[1]) & (df['y_in_pix'] < shape[2])
    coords = df.loc[valid_points, ['z_in_pix', 'x_in_pix', 'y_in_pix']].astype(int).values.T
    labels = df.loc[valid_points, 'Label'].values

    # Set the labels at the coordinates
    label_image[coords[0], coords[1], coords[2]] = labels

    # Apply binary dilation to label_image
    # For each unique label, dilate separately to prevent merging
    for label in tqdm(np.unique(labels), desc='saving points', disable=not verbose):
        binary_mask = label_image == label
        dilated_mask = binary_dilation(binary_mask, structure)
        label_image[dilated_mask] = label

    # Save the dilated label image to a TIFF file
    tifffile.imwrite(output_tiff_path, label_image) 

In [ ]:
import numpy as np
from scipy.ndimage import gaussian_filter


def substract_with_scale(image1, image2, rawtype=np.uint16, uppertype=np.int32):
    image1 = image1.astype(uppertype)
    image2 = image2.astype(uppertype)
    sub = image1 - image2
    min_val = sub.min()
    max_val = sub.max()
    scaled_subtracted_image = (sub - min_val) / (max_val - min_val) * np.iinfo(rawtype).max / 2
    scaled_subtracted_image = scaled_subtracted_image.astype(rawtype)
    return scaled_subtracted_image


def perform_DoG(image, dog_sigma=(0.5,1), enhance=True):
    """
    Apply Difference of Gaussians (DoG) to an image.
    """
    sigma1, sigma2 = dog_sigma
    image1 = gaussian_filter(image, sigma=sigma1)
    image2 = gaussian_filter(image, sigma=sigma2)
    dog = substract_with_scale(image1, image2)
    if enhance: dog = substract_with_scale(1.5 * dog, np.mean(dog))
    return dog

In [ ]:
def scale_tiff(image, dtype=np.uint16):
    """
    Normalize its intensity values to the range dtype / 2.
    """
    # lowerbound = np.percentile(image, 0)
    lowerbound = np.min(image)
    upperbound = np.percentile(image, 99.99)
    # upperbound = np.max(image)
    normalized_image = (image - lowerbound) / (upperbound - lowerbound)
    normalized_image = np.clip(normalized_image, 0, 1)
    scaled_image = (normalized_image * np.iinfo(dtype).max / 2).astype(dtype)
    return scaled_image

# airlocalize data

In [ ]:
import os
from telnetlib import DO
import numpy as np
from skimage.io import imread, imsave
# Assume timtiffread and smooth_image_and_subtract_background7 are previously defined functions

class AirLocalizeData:
    def __init__(self):
        self.reset()

    def reset(self):
        self.srcDir = ''
        self.fList = []
        self.curFile = ''
        self.fileIdx = 0
        self.img = None
        self.smooth = None
        # self.isMovie = False
        # self.nFrames = 0
        # self.curFrame = 0

    def reset_current_file(self):
        """Resets everything but the file list and the isMovie flag."""
        self.curFile = ''
        self.fileIdx = 0
        self.img = None
        self.smooth = None
        # self.nFrames = 0
        # self.curFrame = 0

    def set_flist_from_params(self, params):
        # Placeholder for setting file list from parameters
        # This method will involve file handling and directory searching based on the parameters
        self.srcDir = params['dataFileName']
        self.fList = [_ for _ in os.listdir(params['dataFileName']) if _.endswith('.tif')]

        
    def set_flist(self, flist):
        self.reset()
        if isinstance(flist, list):
            self.fList = flist
        else:
            print('Input file list has wrong format; resetting airlocalize data object.')

    def get_flist(self):
        return self.fList

    def set_file_idx(self, idx):
        if idx > len(self.fList) or idx <= 0:
            print(f'Cannot access file index {idx}; file list has {len(self.fList)} entries.')
            return
        self.reset_current_file()
        self.fileIdx = idx
        self.curFile = self.fList[idx - 1]  # Adjust for zero-based indexing

    def get_file_idx(self):
        return self.fileIdx

    def set_cur_file(self, file_name):
        if file_name not in self.fList:
            print(f'Desired file {file_name} is not part of existing file list; cannot set as current.')
        else:
            idx = self.fList.index(file_name) + 1  # Adjust for one-based indexing in the setter
            self.set_file_idx(idx)

    def get_cur_file(self):
        return self.curFile

    # def set_nframes(self):
    #     # Logic to determine the number of frames; may involve loading the image if not already loaded
    #     pass

    # def set_cur_frame(self, new_frame):
    #     # Logic to update the current frame; involves checking if new_frame is valid
    #     pass

    def is_file_index_img_loaded(self, idx):
        # Checks if the image for the given index is loaded
        pass

    def retrieve_img(self, params):
        # Placeholder for loading an image based on current file index
        self.img = imread(os.path.join(self.srcDir, self.curFile))
        if len(self.img.shape) == 3: self.img = np.transpose(self.img, (1, 2, 0))
        if params['scale']: self.img = scale_tiff(self.img)
        return self.img

    def retrieve_feature_img(self, params):
        if params['featureExtract'] == 'DoG':
            self.smooth = perform_DoG(self.img, dog_sigma=(params['filterLo'], params['filterHi']))
            print(f"smoothed_{self.curFile}")
            imsave(os.path.join(params['saveDirName'], f"smoothed_{self.curFile}"), np.transpose(self.smooth, (2,0,1)))
    # Additional methods as required

# Example usage
# air_data = AirLocalizeData()
# air_data.set_flist(['path/to/image1.tif', 'path/to/image2.tif'])
# air_data.set_file_idx(1)
# print(air_data.get_cur_file())


# main

In [ ]:
import os
import sys
import yaml

import numpy as np
import pandas as pd


def airlocalize(parameters_filename='parameters.yaml'):
    # read parameters from config file
    # parameters_filename = sys.argv[1]

    with open(parameters_filename, 'r') as file:
        params = yaml.safe_load(file)

    al_data = AirLocalizeData()
    al_data.set_flist_from_params(params)

    # load file list from parameters
    try: 
        al_data.get_flist()
        print(f"Found {len(al_data.fList)} files to analyze.")
    except: raise ValueError('could not find files to analyze.')
    
    # set the output dir
    if 'saveDirName' not in params: params['saveDirName'] = os.path.dirname(al_data.flist[0])
    os.makedirs(params['saveDirName'], exist_ok=True)
        
    # Open the log file
    # log_file = open(os.path.join(params['saveDirName'], 'log.txt'), 'a')
    # original_stdout = sys.stdout
    # sys.stdout = log_file

    # save the parameters to the output dir
    with open(os.path.join(params['saveDirName'], 'parameters.yaml'), 'w') as file:
        yaml.dump(params, file)

    # verbose
    verbose = True if params['verbose'] else False

    # loop over files
    for i, file in enumerate(al_data.fList, start=1):
        al_data.set_file_idx(i)
        print(f'Analyzing file: {al_data.curFile}...')

        # Retrieve current image and smooth it
        # overwrite = False if i == 1 else True
        al_data.retrieve_img(params)
        print(al_data.img.shape)

        # Verify that dimensionality agrees with settings
        nd = al_data.img.ndim  # Assuming img is a numpy array or similar
        num_dim_expect = params['numdim']
        if nd != num_dim_expect:
            print(f'Current image has {nd} dimensions but expecting {num_dim_expect}D data; skipping file')
            continue
        
        # Smooth the image and subtract background
        al_data.retrieve_feature_img(params)

        # Pre-detection of local maxima
        spot_candidates = find_isolated_maxima(al_data, params, verbose)
        
        # Detection/quantification
        loc, loc_vars = run_gaussian_fit_on_all_spots_in_image(spot_candidates, al_data, params)

        # Save spot coordinates and detection parameters to a csv file
        df = pd.DataFrame(loc, columns=loc_vars)
        filename = os.path.basename(file).replace('.tif', '')
        df.to_csv(os.path.join(params['saveDirName'], f'{filename}_spots.csv'), index=False)

        # Save the spot image if needed
        if params['output_spots_image']: 
            img_shape_reordered = (al_data.img.shape[2], al_data.img.shape[0], al_data.img.shape[1])
            df['Label'] = [1] * len(df)
            df = df.sort_values(by=['residuals'], ascending=True)
            # Save the spot image
            save_points(df=df, shape=img_shape_reordered, output_tiff_path=os.path.join(params['saveDirName'], f'{filename}_spots.tif'), radius=2, verbose=params['verbose'])

    # Close the log file
    # sys.stdout = original_stdout
    # log_file.close()

In [ ]:
# Example usage
if __name__ == "__main__":
    airlocalize("parameters.yaml")